## 1. Setup & Dependencies

Let's start by installing the required libraries and securely loading our API keys. We'll use `langchain` for agent logic, `mistralai` for the Mistral API, and `python-dotenv` to manage environment variables. For security, never hardcode your API keys in code.

In [1]:
# Install mistral client if not already installed
!pip install mistralai


   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------------------------------------- 0/4 [invoke]
   ---------- ----------------------------- 1/4 [eval-type-backport]
  Attempting uninstall: httpx
   ---------- ----------------------------- 1/4 [eval-type-backport]
    Found existing installation: httpx 0.27.2
   ---------- ----------------------------- 

In [2]:
# Load API keys securely
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from a .env file if present
load_dotenv()

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')

# If not found, prompt the user securely
if not MISTRAL_API_KEY:
    MISTRAL_API_KEY = getpass('Enter your Mistral API key: ')

# Confirm that the key is loaded (do not print the key!)
if MISTRAL_API_KEY:
    print('✅ Mistral API key loaded.')
else:
    raise ValueError('❌ Mistral API key not found. Please set it in your .env file or enter it when prompted.')

✅ Mistral API key loaded.


### Defines the same get_weather function.

Provides a tool schema for function calling.

In [3]:
# Import dependencies
from mistralai import Mistral
import json

# Initialize Mistral client (set your API key as environment variable: MISTRAL_API_KEY)
client = Mistral(api_key=MISTRAL_API_KEY)

# Mock weather function
def get_weather(location: str) -> str:
    """Get current weather for a location (mocked)"""
    return f"Weather in {location}: 72°F, sunny"

# Define the function schema in Mistral's tool format
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"}
            },
            "required": ["location"]
        }
    }
}]


Handles the model’s tool call → executes Python function → returns result.

In [4]:
# Call Mistral model with function-calling capability
response = client.chat.complete(
    model="mistral-large-latest",   # or mistral-small-latest
    messages=[{"role": "user", "content": "What's the weather in Seattle?"}],
    tools=tools,
    tool_choice="auto"
)

print(response)


id='c015de8b53254044bc58e3a7be7e8c62' object='chat.completion' model='mistral-large-latest' usage=UsageInfo(prompt_tokens=68, completion_tokens=13, total_tokens=81, prompt_audio_seconds=Unset()) created=1759335557 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='get_weather', arguments='{"location": "Seattle"}'), id='zCKmsw2Sd', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')]


In [5]:
# Parse the tool call (if the model chose to call get_weather)
message = response.choices[0].message

if message.tool_calls:
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_weather":
            args = json.loads(tool_call.function.arguments)
            result = get_weather(args["location"])
            print("Function result:", result)
else:
    print("LLM response:", message.content)


Function result: Weather in Seattle: 72°F, sunny
